In [1]:
import matplotlib.pyplot as plt
import numpy as np
from filter import filter_encode, filter_decode
from helper import runlength_encode
import keras
from collections import defaultdict

from itertools import chain

from sklearn.cluster import KMeans, MiniBatchKMeans

import multiprocessing as multi
from multiprocessing import Pool

import pickle

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
def freq_vector(arr):
    d = {i: 1 for i in range(256)}
    for i in arr:
        d[i] += 1
    return d

def to_freq(img):
    fil = filter_encode(img, fid=1)
    rle = runlength_encode(fil.ravel())
    return list(freq_vector(rle).values())

In [4]:
rles = []

n_cores = multi.cpu_count()
with Pool(n_cores) as p:
    freq = p.map(to_freq, X)

NameError: name 'X' is not defined

In [5]:
X = [x_train[i, :, :] for i in range(60000)]

n_cores = multi.cpu_count()
with Pool(n_cores) as p:
    freq = p.map(to_freq, X)

print(np.array(freq))

new_X = np.array(freq)

[[ 68  23  30 ...  16  19  13]
 [102  37  40 ...  23  35  31]
 [105  22  26 ...  10  18  21]
 ...
 [ 86  20  14 ...  13  10  18]
 [ 98  37  43 ...  20  19  16]
 [ 44  17   6 ...   8   3  15]]


In [6]:
new_X.shape

(60000, 256)

In [7]:
km = MiniBatchKMeans(n_clusters=256, max_iter=1000)

km.fit(new_X)
km.cluster_centers_

array([[80.        , 25.        , 34.        , ..., 12.        ,
        20.        , 22.        ],
       [58.        , 14.        , 24.        , ..., 12.        ,
        10.        , 16.        ],
       [45.61068702,  1.03816794, 12.06870229, ..., 16.8778626 ,
         2.15267176,  1.44274809],
       ...,
       [80.30769231, 14.92307692,  9.61538462, ...,  5.76923077,
         7.07692308, 11.53846154],
       [91.58830146, 25.99100112, 23.712036  , ..., 11.06299213,
        13.64566929, 25.5928009 ],
       [63.57985758,  3.34384537,  5.00610376, ...,  3.37741607,
         3.37538149,  3.77619532]])

In [8]:
type(km.labels_)

numpy.ndarray

In [15]:
np.sum(new_X[km.labels_ == 0], axis=0).shape

(256,)

In [19]:
from huffman import HuffmanCoding

dicts = []
trees = []

for i in range(256):
    print(i)
    huff = HuffmanCoding()
    label = new_X[km.labels_ == i]

    label_avg = np.average(new_X[km.labels_ == i], axis=0)


    freq = {i: label_avg[i] for i in range(256)}
    huff.create_heap(freq)

    # Code table
    new = huff.build_huffman_tree()
    huff.create_code_dict(new, "")

    dicts.append(huff.code)
    trees.append(huff.tree)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


In [36]:
with open("dicts.pkl", "wb") as f:
    pickle.dump(dicts, f)

with open("trees.pkl", "wb") as f:
    pickle.dump(trees, f)

In [19]:
with open("trees.pkl", "rb") as f:
    trees_loaded = pickle.load(f)

print(trees_loaded)

[<huffman.Node object at 0x14bcce790>, <huffman.Node object at 0x14f0342d0>, <huffman.Node object at 0x14fcdf7d0>, <huffman.Node object at 0x14fd8cad0>, <huffman.Node object at 0x14bcca750>, <huffman.Node object at 0x14ee15910>, <huffman.Node object at 0x1504fad90>, <huffman.Node object at 0x1504bd8d0>, <huffman.Node object at 0x14fcafdd0>, <huffman.Node object at 0x15044d590>, <huffman.Node object at 0x14bcef850>, <huffman.Node object at 0x14f7fa610>, <huffman.Node object at 0x14f548890>, <huffman.Node object at 0x14f5dd710>, <huffman.Node object at 0x14fbd90d0>, <huffman.Node object at 0x14f612710>, <huffman.Node object at 0x14f80ce90>, <huffman.Node object at 0x14f047650>, <huffman.Node object at 0x14bcd6690>, <huffman.Node object at 0x14fab2710>, <huffman.Node object at 0x14f5e3590>, <huffman.Node object at 0x14f938ed0>, <huffman.Node object at 0x14bcea990>, <huffman.Node object at 0x14bce45d0>, <huffman.Node object at 0x14f104ed0>, <huffman.Node object at 0x14f790dd0>, <huffman.No

In [37]:
with open("cluster_centers.pkl", "wb") as f:
    pickle.dump(km.cluster_centers_, f)

km.cluster_centers_

array([[80.        , 25.        , 34.        , ..., 12.        ,
        20.        , 22.        ],
       [58.        , 14.        , 24.        , ..., 12.        ,
        10.        , 16.        ],
       [45.61068702,  1.03816794, 12.06870229, ..., 16.8778626 ,
         2.15267176,  1.44274809],
       ...,
       [80.30769231, 14.92307692,  9.61538462, ...,  5.76923077,
         7.07692308, 11.53846154],
       [91.58830146, 25.99100112, 23.712036  , ..., 11.06299213,
        13.64566929, 25.5928009 ],
       [63.57985758,  3.34384537,  5.00610376, ...,  3.37741607,
         3.37538149,  3.77619532]])

In [40]:
arr = [1, 2, 3, 4, 5, 6]

arr[-5:]

[2, 3, 4, 5, 6]